In [ ]:
import dlt
from newsapi.newsapi_client import NewsApiClient
from dlt.sources.helpers import requests
from datetime import datetime, timedelta
import os

news_api_key = os.environ["NEWS_API_KEY"]


@dlt.resource(table_name="articles_us_en")
def get_articles_us_en(api_key=news_api_key):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(
        language="en", q="United States", sort_by="publishedAt"
    )
    for article in articles["articles"]:
        yield article


# Create and run the pipeline
pipeline = dlt.pipeline(
    pipeline_name="newsapi_articles", destination="athena", dataset_name="newsapi_data"
)

# Run all resources
load_info = pipeline.run([get_articles_us_en])  # Fetching articles for the UK
print(load_info)

In [ ]:
with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM articles_ru_ru") as table:
        issues = table.df()
issues.head()